In [1]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

from data_utility import *
from data_utils import *
from loss import *
from train import *
from deeplab_model.deeplab import *
from dense_vnet.DenseVNet import DenseVNet
from sync_batchnorm import convert_model
import datetime

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
USE_GPU = True
NUM_WORKERS = 12
BATCH_SIZE = 2 

dtype = torch.float32 
# define dtype, float is space efficient than double

if USE_GPU and torch.cuda.is_available():
    
    device = torch.device("cuda:0")
    
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.enabled = True
    # magic flag that accelerate
    
    print('using GPU for training')
else:
    device = torch.device('cpu')
    print('using CPU for training')

using GPU for training


In [3]:
train_dataset = get_full_resolution_dataset(data_type = 'nii_train', 
                transform=transforms.Compose([
                random_affine(90, 15),
                random_filp(0.5)]))
# do data augumentation on train dataset

validation_dataset = get_full_resolution_dataset(data_type = 'nii_test', 
                transform=None)
# no data augumentation on validation dataset

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS)
validation_loader = DataLoader(validation_dataset, batch_size=BATCH_SIZE, shuffle=True,
                    num_workers=NUM_WORKERS) # drop_last
# loaders come with auto batch division and multi-thread acceleration

In [4]:
from vnet import VNet

if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
    
refine_model = VNet(classnum=1)
refine_model = nn.DataParallel(refine_model)
refine_model = convert_model(refine_model)
refine_model = refine_model.to(device, dtype)

optimizer = optim.Adam(refine_model.parameters(), lr=1e-2)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=50)

deeplab = DeepLab(output_stride=16)
deeplab = nn.DataParallel(deeplab)
deeplab = convert_model(deeplab)

checkpoint = torch.load('../deeplab_dilated_save/2019-08-10 09:28:43.844872 epoch: 1160.pth') # best one

deeplab.load_state_dict(checkpoint['state_dict_1'])
deeplab = deeplab.to(device, dtype)

epoch = 0

In [7]:
def get_bboxes(image, label, output, batchsize):
    image_final = torch.zeros((batchsize, 1, 128, 128, 128)).to(device, dtype)
    label_final = torch.zeros((batchsize, 1, 128, 128, 128)).to(device, dtype)
    output_final = torch.zeros((batchsize, 1, 128, 128, 128)).to(device, dtype)
    for b in range(batchsize):
        out = output[b]
        #x, y, z = find_bv_centroid(binarize_output(out))
        x,y,z = loadbvcenter(binarize_output(out))
        x, y, z = np.clip([x, y, z], a_min=64, a_max=192)
        x1 = max(x-64, 0)
        x2 = min(x+64, 256)
        y1 = max(y-64, 0)
        y2 = min(y+64, 256)
        z1 = max(z-64, 0)
        z2 = min(z+64, 256)
        image_final[b] = image[b, :, x1:x2, y1:y2, z1:z2]
        label_final[b] = label[b, :, x1:x2, y1:y2, z1:z2]
        output_final[b] = output[b, :, x1:x2, y1:y2, z1:z2]
    return image_final, label_final, output_final

In [ ]:
epochs = 5000

record = open('train_bv_refine_6.txt','a+')

logger = {'train':[], 'validation_1': []}

min_val = 1

for e in tqdm(range(epoch + 1, epochs)):
# iter over epoches
    epoch_loss = 0
        
    for t, batch in enumerate(train_loader):
    # iter over the train mini batches
        refine_model.train()
        deeplab.eval()
        # Set the model flag to train
        # 1. enable dropout
        # 2. batchnorm behave differently in train and test
        #print(batch['image1_data'])
        image_1 = batch['image1_data'].to(device=device, dtype=dtype)
        image_1 = image_1.view(BATCH_SIZE,1,256,256,256)

        label_1 = batch['image1_label'].to(device=device, dtype=dtype)
        label_1 = label_1.view(BATCH_SIZE,3,256,256,256)

        bv_label = label_1[:, 2, :, :, :]
        bv_label = bv_label.view(BATCH_SIZE,1,256,256,256)

        #original_res = [a[minibatch].item() for a in batch['original_resolution']]

        # Get coarse output from deeplab model from 256 resolution input
        out_coarse = deeplab(image_1)
        out_coarse = out_coarse.view(BATCH_SIZE,3,256,256,256)

        bv_coarse = out_coarse[:, 2, :, :, :]
        bv_coarse = bv_coarse.view(BATCH_SIZE,1,256,256,256)

        bbox_image, bbox_label, bbox_bv = get_bboxes(image_1, bv_label, bv_coarse, BATCH_SIZE)
        #bbox_image, bbox_label, bbox_bv = get_bboxes(image_1, bv_label, bv_coarse, BATCH_SIZE)
        
        bbox_concat = torch.cat([bbox_image, bbox_bv], dim=1)
        
        #show_image_slice(label_1)
        #show_image_slice(bv_label)
        #show_image_slice(bbox_image)
        #show_image_slice(bbox_label)
        #show_image_slice(bbox_bv)

        #print("bbox_concat", bbox_concat.shape)

        del out_coarse
        del image_1
        del bv_coarse
        del label_1
        del bv_label
        del bbox_image
        del bbox_bv

        torch.cuda.empty_cache()

        refine_out = refine_model(bbox_concat)
        # do the inference

        #print(refine_out.shape)
        #print(bbox_bv_label.shape)

        loss = dice_loss(refine_out, bbox_label)
        
        print(loss)
        epoch_loss += loss.item()
        # record minibatch loss to epoch loss
        
        optimizer.zero_grad()
        # set the model parameter gradient to zero
        
        loss.backward()
        # calculate the gradient wrt loss
        optimizer.step()
        # take a gradient descent step
        torch.cuda.empty_cache()
        
    outstr = 'Epoch {0} finished ! Training Loss: {1:.4f}'.format(e, epoch_loss/(t+1)) + '\n'
    
    logger['train'].append(epoch_loss/(t+1))
    
    print(outstr)
    record.write(outstr)
    record.flush()

    if e%2 == 0:
    # do validation every 5 epoches
        deeplab.eval()
        refine_model.eval()
        # set model flag to eval
        # 1. disable dropout
        # 2. batchnorm behave differs

        with torch.no_grad():
        # stop taking gradient
        
            #valloss_4 = 0
            #valloss_2 = 0
            valloss_1 = 0
            
            for v, vbatch in enumerate(validation_loader):
                image_1 = vbatch['image1_data'].to(device=device, dtype=dtype)
                image_1 = image_1.view(BATCH_SIZE,1,256,256,256)

                label_1 = vbatch['image1_label'].to(device=device, dtype=dtype)
                label_1 = label_1.view(BATCH_SIZE,3,256,256,256)

                bv_label = label_1[:, 2, :, :, :]
                bv_label = bv_label.view(BATCH_SIZE,1,256,256,256)

                #original_res = [a[minibatch].item() for a in batch['original_resolution']]

                # Get coarse output from deeplab model from 256 resolution input
                out_coarse = deeplab(image_1)
                out_coarse = out_coarse.view(BATCH_SIZE,3,256,256,256)

                bv_coarse = out_coarse[:, 2, :, :, :]
                bv_coarse = bv_coarse.view(BATCH_SIZE,1,256,256,256)

                bbox_image, bbox_label, bbox_bv = get_bboxes(image_1, bv_label, bv_coarse, BATCH_SIZE)

                bbox_concat = torch.cat([bbox_image, bbox_bv], dim=1)

                #show_image_slice(label)
                #show_image_slice(bbox_image)
                #show_image_slice(bbox_label)
                #show_image_slice(bbox_bv)

                #print("bbox_concat", bbox_concat.shape)
                #seg_image_concat = torch.cat([bv_coarse, image_1], dim=1)

                del out_coarse
                del image_1
                del bv_coarse
                del label_1
                del bv_label
                del bbox_image
                del bbox_bv

                torch.cuda.empty_cache()

                refine_out = refine_model(bbox_concat)
                        
                loss = dice_loss(refine_out, bbox_label)
                
                print(loss)
            
                # calculate loss
                valloss_1 += loss.item()
                
                torch.cuda.empty_cache()
                
            
            avg_val_loss = (valloss_1 / (v+1))
            outstr = '------- 1st valloss={0:.4f}'\
                .format(avg_val_loss) + '\n'
            
            logger['validation_1'].append(avg_val_loss)
            scheduler.step(avg_val_loss)
            
            if avg_val_loss < min_val:
                save_1('refine_bv6_save', refine_model, optimizer, logger, e, scheduler)
            elif e % 10 == 0:
                save_1('refine_bv6_save', refine_model, optimizer, logger, e, scheduler)
            
            torch.cuda.empty_cache()
            
            print(outstr)
            record.write(outstr)
            record.flush()
    


record.close()

  0%|          | 0/4999 [00:00<?, ?it/s]

tensor(0.9670, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9658, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9776, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9363, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9670, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9883, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9528, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9753, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9646, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9875, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9754, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9682, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9625, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9725, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9623, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9389, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9807, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9608, device='cuda:0',

  0%|          | 1/4999 [16:35<1382:01:19, 995.45s/it]

Epoch 1 finished ! Training Loss: 0.9360

tensor(0.8582, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7597, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7478, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7567, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.8759, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.8312, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.8983, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7386, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7208, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7124, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7377, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7748, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6790, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6369, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7189, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9583, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.7108, device='cuda:0', grad_fn=<RsubB

  0%|          | 2/4999 [32:55<1375:21:02, 990.85s/it]

...

Checkpoint 4 saved !
------- 1st valloss=0.2065

tensor(0.2631, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.1122, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.3024, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.1757, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.1576, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6171, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.0999, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9926, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.1273, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.0778, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.2183, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.1500, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5739, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.1804, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5412, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5639, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5506, device='cuda:0', grad_fn

  0%|          | 5/4999 [1:19:14<1315:55:51, 948.61s/it]

Epoch 5 finished ! Training Loss: 0.3679

tensor(0.1043, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.1257, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.1351, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5918, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9955, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.0901, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5592, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.3231, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.2125, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.2325, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.5309, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.1834, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.1125, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.9929, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.1641, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.6214, device='cuda:0', grad_fn=<RsubBackward1>)
tensor(0.1398, device='cuda:0', grad_fn=<RsubB

  0%|          | 6/4999 [1:35:23<1324:23:49, 954.90s/it]

Checkpoint 6 saved !
------- 1st valloss=0.1686



In [ ]:
deeplab.eval()

with torch.no_grad():
    
    bgloss = 0
    bdloss = 0
    bvloss = 0
    
    for v, vbatch in tqdm(enumerate(validation_loader)):
        # move data to device, convert dtype to desirable dtype
        image_1 = vbatch['image1_data'].to(device=device, dtype=dtype)
        label_1 = vbatch['image1_label'].to(device=device, dtype=dtype)

        output = deeplab(image_1)
        # do the inference
        output_numpy = output.cpu().numpy()
        
        
        #out_1 = torch.round(output)
        out_1 = torch.from_numpy((output_numpy == output_numpy.max(axis=1)[:, None]).astype(int)).to(device=device, dtype=dtype)
        loss_1 = dice_loss_3(out_1, label_1)

        bg, bd, bv = dice_loss_3_debug(out_1, label_1)
        # calculate loss
        print(bg.item(), bd.item(), bv.item(), loss_1.item())
        bgloss += bg.item()
        bdloss += bd.item()
        bvloss += bv.item()
        
        if bv.item() >= 0.2 or bd.item() >= 0.1:
            show_image_slice(image_1)
            show_image_slice(label_1)
            show_image_slice(output)

    outstr = '------- background loss = {0:.4f}, body loss = {1:.4f}, bv loss = {2:.4f}'\
        .format(bgloss/(v+1), bdloss/(v+1), bvloss/(v+1)) + '\n'
    print(outstr)